In [1]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json -P ./data/ShapeNet/ShapeNetCore.v1
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/r2n2/r2n2_synset_dict.json -P ./data/ShapeNet/

--2021-05-06 00:37:32--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1510 (1.5K) [text/plain]
Saving to: ‘./data/ShapeNet/ShapeNetCore.v1/shapenet_synset_dict_v1.json.3’

shapenet_synset_dic 100%[===================>]   1.47K  --.-KB/s    in 0s      

2021-05-06 00:37:32 (5.27 MB/s) - ‘./data/ShapeNet/ShapeNetCore.v1/shapenet_synset_dict_v1.json.3’ saved [1510/1510]

--2021-05-06 00:37:32--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/pytorch3d/datasets/r2n2/r2n2_synset_dict.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecti

In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import numpy as np
import torch
import pytorch3d

from pytorch3d.datasets import (
    R2N2,
    ShapeNetCore,
    collate_batched_meshes,
    collate_batched_R2N2,
    render_cubified_voxels,
)
from pytorch3d.renderer import (
    OpenGLPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    TexturesVertex,
    look_at_view_transform,
)

from pytorch3d.structures import Meshes, Pointclouds
from torch.utils.data import DataLoader
#from src.utils.plot_image_grid import image_grid

#sys.path.insert(0, '../src')
#from autoencoder import *

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
    
#SHAPENET_PATH = "/iris/u/clin/code/cs231n-project/data/ShapeNet/ShapeNetCore.v1/"
SHAPENET_PATH = "./data/ShapeNet/ShapeNetCore.v1/"
shapenet_dataset = ShapeNetCore(SHAPENET_PATH)
#shapenet_dataset = ShapeNetCore(SHAPENET_PATH, synsets=['cabinet'])

FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/lib/python3.8/site-packages/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json'

In [ ]:
shapenet_dataset.synset_num_models

In [ ]:
R2N2_PATH = "/iris/u/clin/code/cs231n-project/data/ShapeNet/"
SPLITS_PATH = "/iris/u/clin/code/cs231n-project/data/ShapeNet/ShapeNetRendering/pix2mesh_splits_val05.json"
r2n2_dataset = R2N2("train", SHAPENET_PATH, R2N2_PATH, SPLITS_PATH, return_voxels=False)

In [ ]:
r2n2_dataset.synset_num_models

In [26]:
import os
import argparse

import torch
from torch.utils.data import DataLoader
from pytorch3d.datasets import R2N2, collate_batched_R2N2

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

from autoencoder import AutoEncoder


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--gpus', default=1)
    parser.add_argument('--data_dir', type=str, default='/home/data')
    args = parser.parse_args()

    synsets = ['cabinet']  # use one synset for easy prototyping
    shapenet_path = os.path.join(args.data_dir, 'ShapeNet/ShapeNetCore.v1')
    r2n2_path = os.path.join(args.data_dir, 'ShapeNet')
    r2n2_splits = os.path.join(args.data_dir, 'ShapeNet/ShapeNetRendering/pix2mesh_splits_val05.json')

    train_set = R2N2("train", shapenet_path, r2n2_path, r2n2_splits, return_voxels=False)
    val_set = R2N2("val", shapenet_path, r2n2_path, r2n2_splits, return_voxels=False)

    train_loader = DataLoader(train_set, batch_size=4, collate_fn=collate_batched_R2N2, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=4, collate_fn=collate_batched_R2N2, num_workers=2)

    ae = AutoEncoder()  # TODO pass in whatever args are used in the constructor
    wandb_logger = WandbLogger(name='test', project='3d-recon', entity='3drecon2')
    trainer = pl.Trainer(gpus=args.gpus, logger=wandb_logger)
    trainer.fit(ae, train_loader, val_loader)

usage: ipykernel_launcher.py [-h] [--gpus GPUS] [--data_dir DATA_DIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/carolinechoi/Library/Jupyter/runtime/kernel-77e7cdbd-ab19-4dba-9340-25d83bed99d3.json


SystemExit: 2